[View in Colaboratory](https://colab.research.google.com/github/ShriPunta/Netflix-Graph-Dataset-Project/blob/master/ParsingTheCombinedFile.ipynb)

In [0]:
!pip install -U -q PyDrive


**Setup all the Authentication for PyDrive**

In [0]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build


In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


---


** *Method*: To read the contents of a local Google File**



In [0]:
def read_drive_file_into_variable(file_id_to_read):
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id_to_read)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    status, done = downloader.next_chunk()
    prog = int(status.progress() * 100)
    print("Download "+str(prog))
  
  #Set the pointer to the start
  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()))
  
  #Read Everthing into a variable called "View", its in a "Bytes" datatype
  view = downloaded.read1(-1)
  #len(view)
  
  #decode Bytes to String format
  decoded = view.decode(encoding="utf-8")
  #type(decoded)
  
  #Convert this continous string to List for each new line
  variable_to_set = decoded.splitlines()
  #type(splitted)
  
  return variable_to_set
  
  

---

**Read the main two files into variables**

In [0]:
movie_Title_file_id = '0B8qgJkz0ynl8czBBNG9qZ2JPeW9RaWVuZktobUE1b29qaER3'
rating_file_id = '1iABtudmoCPxcFYiYQ0cQwfcQCZtfHsHd'
test_file_id = '11QhP0HwV7x6huJX3-JqJzGv7shMiZVPH'

In [9]:
key_file_list = read_drive_file_into_variable(movie_Title_file_id)


Download 100


In [10]:
rating_file_list = read_drive_file_into_variable(rating_file_id)

Download 21
Download 42
Download 63
Download 84
Download 100


In [0]:
#test_file_list = read_drive_file_into_variable(test_file_id)

---

**Convert the key file into a dataframe**

In [0]:
#Read the key file into a Panda dataframe
#key_file_df = pd.DataFrame([sub.split(",") for sub in key_file_list],columns = ["MovieId","Year","Name","Genre1","Genre2","Genre3"])
#key_file_df.head()

**Clean the dataframe**

In [0]:
# key_file_df['Genre'] = key_file_df[['Genre1','Genre2','Genre3']].apply(lambda x: ''.join(x), axis=1)
# key_file_df.head()

def clean_the_movie_key_df(key_file_df):
  #Combine the Genres into a single column
  key_file_df['Genre'] = key_file_df['Genre1'] + ',' + key_file_df['Genre2'] + ','+ key_file_df['Genre3']

  #Clean the column by removing double quotes
  for i, col in enumerate(key_file_df.columns):
      key_file_df.iloc[:,6] = key_file_df.iloc[:,6].str.replace('"', '')

  #Drop the unnecessary columns
  key_file_df.drop(['Genre1','Genre2','Genre3'], axis=1, inplace=True)

  #Set MovieId as the Index
  key_file_df.set_index('MovieId',inplace=True)

  return key_file_df


In [0]:
#key_file_df = clean_the_movie_key_df(key_file_df)

In [23]:
#key_file_df.head()
  
  

,Year,Name,Genre
MovieId,,,
1,2003,Dinosaur Planet,"Animation,Documentary,Family"
3,1997,Character,"Crime,Drama,Mystery"
6,1997,Sick,NaN
7,1992,8 Man,NaN
10,2001,Fighter,NaN


---

**Convert rating File into dataframe**

In [16]:
#Read the key file into a Panda dataframe
test_file_df = pd.DataFrame([sub.split(",") for sub in rating_file_list],columns = ["UserId","Rating","DateWatched"])

#Initialize a blank column called MovieId ; to be filled latter
test_file_df["MovieId"] = ''

test_file_df.head()


,UserId,Rating,DateWatched,MovieId
0,1:,None,None,
1,1488844,3,2005-09-06,
2,822109,5,2005-05-13,
3,885013,4,2005-10-19,
4,30878,4,2005-12-26,


---

In [22]:
#We will store the rows to drop in this
rows_to_drop = []
currentMovieId = 0

#Iterate over the dataframe to split it
for index, row in test_file_df.iterrows():
  if row['UserId'].find(':') != -1:
    #If entered it means that there is a colon on the row and it is a movie id
    #row[0][0] gives the first character on that row
    currentMovieId = int(row[0][0])
    rows_to_drop.append(index)
  else:
    #print("Not")
    #Assign the movie id to the consecutive rows
    test_file_df.iloc[index][3] = currentMovieId

KeyboardInterrupt: ignored

In [23]:
#We get a dataframe with the movie id generated, now we need to filter out the rows
#test_file_df.head(50)
len(test_file_df.index)

24058263

In [21]:
#Drop the rows which had the movie Id and the columns to get a seamless dataframe
test_file_df.drop(test_file_df.index[rows_to_drop],inplace=True)
test_file_df.head()

NameError: ignored

In [0]:
#Convert the column to datetime and keep only the year
test_file_df['YearWatched'] = pd.to_datetime(test_file_df['DateWatched']).dt.year

In [20]:
test_file_df.head()

,UserId,Rating,DateWatched,MovieId,YearWatched
1,1488844,3,2005-09-06,1,2005
2,822109,5,2005-05-13,1,2005
3,885013,4,2005-10-19,1,2005
4,30878,4,2005-12-26,1,2005
6,1488844,3,2005-09-06,2,2005


---
 ** *Method*: To see the GPU usage**

In [0]:
def checkGPU():
  # memory footprint support libraries/code
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
  !pip install gputil
  !pip install psutil
  !pip install humanize
  import psutil
  import humanize
  import os
  import GPUtil as GPU
  GPUs = GPU.getGPUs()
  # XXX: only one GPU on Colab and isn’t guaranteed
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  


---

In [17]:
checkGPU()

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Gen RAM Free: 2.1 GB  I Proc size: 11.2 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB
